In [17]:
import cv2
import glob
import os
from skin_segmentation import *
import pandas as pd
from converter import RGB_TO_HSI

annotation_folder = "./Dataset/full_coco/combined_masks_by_category/"
image_folder = "./Dataset/full_coco/images/"

#df = pd.DataFrame(columns = ['h', 's', 'i', 'burn_skin'])
df = pd.read_csv("./Dataset/pixelwise_skin_and_burn_trial.csv")
completed_files = []
with open('./Dataset/completed_files_trial.txt', 'r') as f:
    for item in f:
        completed_files.append(item.rstrip())
print("Files already completed:", completed_files)
print("Imported")

Files already completed: ['./Dataset/full_coco/images/img10.jpg', './Dataset/full_coco/images/img100.jpg', './Dataset/full_coco/images/img101.jpg', './Dataset/full_coco/images/img102.jpg', './Dataset/full_coco/images/img103.jpg', './Dataset/full_coco/images/img104.jpg', './Dataset/full_coco/images/img105.jpg', './Dataset/full_coco/images/img106.jpg', './Dataset/full_coco/images/img107.jpg', './Dataset/full_coco/images/img108.jpg', './Dataset/full_coco/images/img109.jpg', './Dataset/full_coco/images/img11.jpg', './Dataset/full_coco/images/img110.jpg', './Dataset/full_coco/images/img111.jpg', './Dataset/full_coco/images/img112.jpg', './Dataset/full_coco/images/img113.jpg', './Dataset/full_coco/images/img114.jpg', './Dataset/full_coco/images/img115.jpg', './Dataset/full_coco/images/img116.jpg', './Dataset/full_coco/images/img117.jpg', './Dataset/full_coco/images/img118.jpg', './Dataset/full_coco/images/img119.jpg', './Dataset/full_coco/images/img12.jpg', './Dataset/full_coco/images/img120

In [18]:
file_num = 0
file_list = glob.glob(image_folder + "*.jpg")
file_list.sort()

print("Files to process:", len(file_list))
for filename in file_list:
    #print("Got", filename)
    if filename not in completed_files:
        base_name = os.path.basename(filename)
        skin_name = annotation_folder + base_name.replace(".jpg", "_category1.jpg")
        burn_name = annotation_folder + base_name.replace(".jpg", "_category2.jpg")

        if os.path.exists(skin_name) and os.path.exists(burn_name):
            image = load_rgb_image(filename)
            hsi_image = RGB_TO_HSI(image)
            mask1 = cv2.imread(skin_name, cv2.IMREAD_GRAYSCALE) / 255
            mask2 = cv2.imread(burn_name, cv2.IMREAD_GRAYSCALE) / 255
            print("Starting:", base_name)
            pixel_list = []
            for col in range(0, image.shape[1]):
                for row in range(0, image.shape[0]):
                    h, s, i = hsi_image[row,col]
                    if mask2[row,col] > 0:
                        pixel_list.append({'h':h, 's':s, 'i':i, 'burn_skin':2})
                    elif mask1[row,col] > 0:
                        pixel_list.append({'h':h, 's':s, 'i':i, 'burn_skin':1})
                    else:
                        pixel_list.append({'h':h, 's':s, 'i':i, 'burn_skin':0})
            new_df = pd.DataFrame(pixel_list)
            df = pd.concat([df, new_df], axis=0, ignore_index=True)
            completed_files.append(filename)
            df.to_csv("./Dataset/pixelwise_skin_and_burn_trial.csv",index=False)
            with open('./Dataset/completed_files_trial.txt', 'w') as f:
                for item in completed_files:
                    f.write(item + "\n" )
        else:
            print("Skipping due to lack of category")
    else:
        print("Already completed")
    if file_num % 10 == 0:
        print("\n****Files Completed:", file_num, "****\n")
    file_num += 1

Files to process: 228
Skipping due to lack of category

****Files Completed: 0 ****

Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed

****Files Completed: 10 ****

Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Already completed

****Files Completed: 20 ****

Already completed
Already completed
Already completed
Already completed
Already completed
Already completed
Skipping due to lack of category
Already completed
Skipping due to lack of category
Already completed

****Files Completed: 30 ****

Skipping due to lack of category
Already completed
Skipping due to lack of category
Already completed
Skipping due to lack of category
Already completed
Skipping due to lack of category
Already completed
Skipping due to lack of category
Already complet

In [19]:
print("Files Completed:", len(completed_files))
print("Total Entries Should Be:", (len(completed_files) * 250 * 250))
print("Actual Entries:", len(df))
df.to_csv("./Dataset/pixelwise_skin_and_burn.csv",index=False)


Files Completed: 197
Total Entries Should Be: 12312500
Actual Entries: 5812500
